In [1]:
module O

abstract type Mode end

struct Grid <: Mode end
read_grid(filename) = "read_grid: $filename"
readfile(filename, mode::Grid) = read_grid(filename)

DefaultMode() = Grid()
readfile(filename) = readfile(filename, DefaultMode())

struct PL3D <: Mode end
read_pl3d(filename) = "read_pl3d: $filename"
readfile(filename, mode::PL3D) = read_pl3d(filename)

struct Restart <: Mode end
read_restart(filename) = "read_restart: $filename"
readfile(filename, mode::Restart) = read_restart(filename)

struct Header <: Mode end
read_header(filename) = "read_header: $filename"
readfile(filename, mode::Header) = read_header(filename)

end

Main.O

In [2]:
filename = "foo.ext"
@eval @show O.readfile($filename)
for mode in (O.Grid(), O.PL3D(), O.Restart(), O.Header())
    @eval @show O.readfile($filename, $mode)
end

O.readfile("foo.ext") = "read_grid: foo.ext"
O.readfile("foo.ext", Main.O.Grid()) = "read_grid: foo.ext"
O.readfile("foo.ext", Main.O.PL3D()) = "read_pl3d: foo.ext"
O.readfile("foo.ext", Main.O.Restart()) = "read_restart: foo.ext"
O.readfile("foo.ext", Main.O.Header()) = "read_header: foo.ext"


上のようなスタイルでコードを書いておくと, モジュール O のコードを変更することなく, 他のモジュールで `O.readfile` メソッドを自由に拡張できるようになる.  このような拡張はJulia言語のエコシステムでは最も普通に行われていることである.  特に `Base.show` メソッドの独自拡張の頻度は極めて多い.

In [3]:
module P

using ..O

struct ModelP <: O.Mode end
read_modelp(filename) = "read_modelp: $filename"
O.readfile(filename, mode::ModelP) = read_modelp(filename)

end

Main.P

In [4]:
filename = "foo.ext"
@eval @show O.readfile($filename)
for mode in (O.Grid(), O.PL3D(), O.Restart(), O.Header(), P.ModelP())
    @eval @show O.readfile($filename, $mode)
end

O.readfile("foo.ext") = "read_grid: foo.ext"
O.readfile("foo.ext", Main.O.Grid()) = "read_grid: foo.ext"
O.readfile("foo.ext", Main.O.PL3D()) = "read_pl3d: foo.ext"
O.readfile("foo.ext", Main.O.Restart()) = "read_restart: foo.ext"
O.readfile("foo.ext", Main.O.Header()) = "read_header: foo.ext"
O.readfile("foo.ext", Main.P.ModelP()) = "read_modelp: foo.ext"


## メモ1

```julia
readfile(filename::String, mode::Grid) = read_grid(filename)
```

と書くことはやめた方がよい. `filename::String` の部分に問題がある. このように書いてはいけない理由は文字列のように振る舞うが `String` 型でないオブジェクトが沢山あるからである. 例えば

In [5]:
m = match(r"foo\d+.ext", "/bar/foo123.ext").match

"foo123.ext"

In [6]:
my_readfile(filename::String, mode::O.Grid) = O.read_grid(filename)
my_readfile(m, O.Grid())

LoadError: MethodError: no method matching my_readfile(::SubString{String}, ::Main.O.Grid)
[0mClosest candidates are:
[0m  my_readfile([91m::String[39m, ::Main.O.Grid) at In[6]:1

In [7]:
typeof(m)

SubString{String}

In [8]:
m isa String

false

函数の引数の型を書くとすれば

```
readfile(filename::AbstractString, mode::Grid) = read_grid(filename)
```

と `String` ではなく `AbstractString` を使うべき.

In [9]:
m isa AbstractString

true

In [10]:
my_readfile(filename::AbstractString, mode::O.Grid) = O.read_grid(filename)
my_readfile(m, O.Grid())

"read_grid: foo123.ext"

## メモ2

さらに module O の中で

```julia
readfile(filename, mode::Mode) = println("Not implemented for ", typeof(mode))
```

のように定義すると, デバッグが困難になる原因になるので注意が必要. 詳しくは

* https://www.oxinabox.net/2020/04/19/Julia-Antipatterns.html#notimplemented-exceptions

を参照.

たとえば, 函数の引数の型注釈を

```
readfile(filename::String, mode::Grid) = read_grid(filename)
```

と書いているときに(これはメモ1で述べたようにやめた方がよい), 上のように not implemented エラーを出すようにしてあると, 以下のようなことが起こって困惑してしまうことになる.

In [11]:
module O1

abstract type Mode end
readfile(filename, mode::Mode) = println("Not implemented for ", typeof(mode))

struct Grid <: Mode end
read_grid(filename) = "read_grid: $filename"
readfile(filename::String, mode::Grid) = read_grid(filename)

end

Main.O1

In [12]:
O1.readfile(m, O1.Grid())

Not implemented for Main.O1.Grid


In [13]:
m

"foo123.ext"

In [14]:
typeof(m)

SubString{String}

以下のように Not implemented エラーのために函数を定義していない方がエラーメッセージが分かり易い.

In [15]:
module O2

abstract type Mode end

struct Grid <: Mode end
read_grid(filename) = "read_grid: $filename"
readfile(filename::String, mode::Grid) = read_grid(filename)

end

Main.O2

In [16]:
@show m
O2.readfile(m, O2.Grid())

m = "foo123.ext"


LoadError: MethodError: no method matching readfile(::SubString{String}, ::Main.O2.Grid)
[0mClosest candidates are:
[0m  readfile([91m::String[39m, ::Main.O2.Grid) at In[15]:7